In [112]:
import numpy as np
import torch
import math
from torch.quasirandom import SobolEngine
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from dataclasses import dataclass
import gpytorch
from botorch.models import SingleTaskGP
from botorch.optim import optimize_acqf
from botorch.acquisition import UpperConfidenceBound, ExpectedImprovement
from dataclasses import dataclass

In [160]:
# we define a dataclass for our state
@dataclass
class TurboState:
    dim: int # dimension of the problem, aka input dimension
    batch_size: int = 1 # we could do batch optimization, but the capstone only does one query at a time
    length: float = 0.3 # the length of the current trust region
    length_min: float = 0.01 # minimum length for the trust region
    length_max: float = 1.0 # maximum length for the trust region
    failure_counter: int = 0 # initialize counter of the number of failures to improve on the best observation
    failure_tolerance: int = float("nan")  # Note: Post-initialized
    success_counter: int = 0 # initialize counter of the number of success to improve on the best observation
    success_tolerance: int = 5  # Note: The original paper uses 3, this is the number of successes in a row needed to expand the region
    best_value: float = -float("inf") # best value so far, initialized to be the infimum
    restart_triggered: bool = False 

    def __post_init__(self):
        self.failure_tolerance = math.ceil(
            max([4.0 / self.batch_size, float(self.dim) / self.batch_size]) # number of failures needed in a row to shrink the trust region
        )


def update_state(state, Y_next):
    # count if a success, otherwise a failure
    if max(Y_next) > state.best_value + 1e-3 * math.fabs(state.best_value):
        state.success_counter += 1
        state.failure_counter = 0
    else:
        state.success_counter = 0
        state.failure_counter += 1
    # check if we need to expand or shrink the trust region
    if state.success_counter == state.success_tolerance:  # Expand trust region
        state.length = min(2.0 * state.length, state.length_max)
        state.success_counter = 0
    elif state.failure_counter == state.failure_tolerance:  # Shrink trust region
        state.length /= 2.0
        state.failure_counter = 0
    # set the best value if we got a new observation
    state.best_value = max(state.best_value, max(Y_next).item())
    if state.length < state.length_min:
        state.restart_triggered = True
    return state

In [161]:
# Define model using SingleTaskGP
def create_model(train_x, train_y):
    train_y = train_y.view(-1, 1)  # Ensure train_y has the correct shape
    model = SingleTaskGP(train_x, train_y)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(model.likelihood, model)
    return model, mll

import gpytorch.settings as settings

def fit_gpytorch_model(mll):
    mll.train()
    optimizer = torch.optim.Adam(mll.parameters(), lr=0.05)
    training_iter = 300

    for i in range(training_iter):
        optimizer.zero_grad()
        output = mll.model(mll.model.train_inputs[0])
        with settings.use_toeplitz(False), settings.max_cholesky_size(2000), settings.cholesky_jitter(1e-4):
            loss = -mll(output, mll.model.train_targets)
        loss.backward()
        optimizer.step()

        #if i % 50 == 49:
        #    print(f'Iter {i+1}/{training_iter} - Loss: {loss.item():.3f}   lengthscale: {mll.model.covar_module.base_kernel.lengthscale.detach()}   noise: {mll.model.likelihood.noise.item():.3f}')
            
# we use the model given in the tutorial, we also add the hyper-parameter training as a method
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        # set a constant mean
        self.mean_module = gpytorch.means.ConstantMean()
        # use a simple RBF kernel with constant scaling
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=train_x.shape[1]))
        # set number of hyper-parameter training iterations
        self.training_iter = 300
        self.num_outputs = 1  # Add this line

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    

In [162]:
def generate_batch(
    state,
    model,  # GP model
    X,  # Evaluated points on the domain [0, 1]^d
    Y,  # Function values
    batch_size=1,  # fix batch size to 1
    n_candidates=None,  # Number of candidates for Thompson sampling
    num_restarts=10,
    raw_samples=512,
    acqf="ts",  # "ei", "ucb", or "ts"
):
    assert acqf in ("ts", "ei", "ucb","ucb0")
    assert X.min() >= 0.0 and X.max() <= 1.0 and torch.all(torch.isfinite(Y))
    if n_candidates is None:
        n_candidates = min(10000, max(2000, 200 * X.shape[-1]))

    # Scale the trust region to be proportional to the lengthscales
    x_center = X[Y.argmax(), :].clone()
    weights = model.covar_module.base_kernel.lengthscale.squeeze().detach()
    weights = weights / weights.mean()
    weights = weights / torch.prod(weights.pow(1.0 / len(weights)))
    tr_lb = torch.clamp(x_center - weights * state.length / 2.0, 0.0, 1.0)
    tr_ub = torch.clamp(x_center + weights * state.length / 2.0, 0.0, 1.0)
    
    dim = X.shape[-1]
    sobol = SobolEngine(dim, scramble=True)
    pert = sobol.draw(n_candidates)
    pert = tr_lb + (tr_ub - tr_lb) * pert

    # Create a perturbation mask
    prob_perturb = min(20.0 / dim, 1.0)
    mask = (
        torch.rand(n_candidates, dim)
        <= prob_perturb
    )
    ind = torch.where(mask.sum(dim=1) == 0)[0]
    mask[ind, torch.randint(0, dim - 1, size=(len(ind),))] = 1

    # Create candidate points from the perturbations and the mask        
    X_cand = x_center.expand(n_candidates, dim).clone()
    X_cand[mask] = pert[mask]
    X_cand.requires_grad_(True)  # Ensure requires_grad is True for optimization

    model.eval()
    if acqf == "ts":
        # Sample on the candidate points using Thompson Sampling
        posterior_distribution = model.posterior(X_cand)
        posterior_sample = posterior_distribution.sample()
        X_next_idx = torch.argmax(posterior_sample)
        X_next = X_cand[X_next_idx]
    else:
        if acqf == "ei":
            # Expected Improvement acquisition function
            acq_function = ExpectedImprovement(model=model, best_f=Y.max())
        elif acqf == "ucb":
            # Upper Confidence Bound acquisition function
            acq_function = UpperConfidenceBound(model=model, beta=1.96)
        elif acqf == "ucb0":
            # Upper Confidence Bound acquisition function
            acq_function = UpperConfidenceBound(model=model, beta=0.1)
            
        # Optimize the acquisition function
        bounds = torch.stack([torch.zeros(dim), torch.ones(dim)]).to(X_cand.device)
        X_next, _ = optimize_acqf(
            acq_function,
            bounds=bounds,
            q=batch_size,
            num_restarts=num_restarts,
            raw_samples=raw_samples,
        )
        X_next.requires_grad_(True)  # Ensure X_next has requires_grad=True

    return X_next

In [163]:
def next_query_via_TurBO(train_x, train_y, turbo_state, training_iter=300, verbose=False):
    model, mll = create_model(train_x, train_y)
    fit_gpytorch_model(mll)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

    for i in range(training_iter):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        optimizer.step()

        #if verbose and (i % 50 == 49):
        #    print(f'Iter {i+1}/{training_iter} - Loss: {loss.item():.3f}   lengthscale: {model.covar_module.base_kernel.lengthscale.detach()}   noise: {model.likelihood.noise.item():.3f}')

    return generate_batch(turbo_state, model=model, X=train_x, Y=train_y)

### Run!!!

In [164]:
# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

In [165]:
train_x = torch.from_numpy(np.load('initial_data/function_8/initial_inputs.npy')).to(torch.float32)
train_y = torch.from_numpy(np.load('initial_data/function_8/initial_outputs.npy')).to(torch.float32)

state = TurboState(dim = 2, best_value = torch.max(train_y).float())

In [166]:

# New data to append
new_inputs = [
    [0.111111, 0.111111, 0.111111, 0.111111, 0.111111, 0.111111, 0.111111, 0.111111],
    [0.5, 1.0e-06, 0.25, 0.75, 0.25, 0.25, 0.25, 0.75],
    [0.15, 0.15, 0.35, 0.55, 0.15, 0.725, 0.55, 0.95],
    [0.333334, 0.5, 0.5, 0.333334, 0.333334, 0.333334, 0.666666, 0.5],
    [0.200001, 0.200001, 0.4, 0.4, 0.200001, 0.799999, 0.6, 0.799999],
    [0.157895, 0.052632, 0.157895, 0.333333, 0.210527, 0.333333, 0.31579, 0.5],
    [0.25, 0.052632, 0.210527, 0.5, 0.210527, 0.5, 0.210527, 0.75],
    [0.052632, 0.052632, 0.157895, 0.511111, 0.263158, 0.499999, 0.526316, 0.749999],
    [1.0e-06, 0.2, 0.25, 1.0e-06, 0.75, 0.5, 0.25, 1.0e-06],
    [1.0e-06, 0.33334, 1.0e-06, 1.0e-06, 0.999999, 0.5, 1.0e-06, 0.833333],
    [0.052632, 0.473684, 1.0e-06, 1.0e-06, 0.526316, 1.0e-06, 0.105264, 1.0e-06],
    [0.166667, 0.333334, 0.166667, 0.166667, 0.666666, 0.5, 0.166667, 0.5]
]

new_outputs = [
    9.56743,
    9.033300299999,
    9.170675,
    8.751078688886,
    9.1023007399954,
    9.7185386826085,
    9.6273548350665,
    9.4939825701369,
    9.8695508199969,
    9.7677466955466,
    9.476142128,
    9.941077316
]


In [167]:
# Convert new data to tensors
new_train_x = torch.tensor(new_inputs, dtype=torch.float32)
new_train_y = torch.tensor(new_outputs, dtype=torch.float32)

# Append the new data to the existing tensors
train_x = torch.cat((train_x, new_train_x), dim=0)
train_y = torch.cat((train_y, new_train_y), dim=0)

# Ensure train_x is within [0, 1] range
scaler_x = MinMaxScaler()
train_x = torch.tensor(scaler_x.fit_transform(train_x.numpy()), dtype=torch.float32)

# Check for non-finite values in train_y and handle them
if not torch.all(torch.isfinite(train_y)):
    print("Non-finite values detected in train_y")
    finite_mask = torch.isfinite(train_y)
    train_x = train_x[finite_mask]
    train_y = train_y[finite_mask]

# Ensure train_y is properly scaled (Standardization)
scaler_y = StandardScaler()
train_y = torch.tensor(scaler_y.fit_transform(train_y.reshape(-1, 1)).flatten(), dtype=torch.float32)

# Convert to double precision for BoTorch
train_x = train_x.double()
train_y = train_y.double()

In [168]:
state = TurboState(dim=2, best_value=torch.max(train_y).float())
print(state)

TurboState(dim=2, batch_size=1, length=0.3, length_min=0.01, length_max=1.0, failure_counter=0, failure_tolerance=4, success_counter=0, success_tolerance=5, best_value=tensor(3.3213), restart_triggered=False)


In [169]:
# Create and train the model
model, mll = create_model(train_x, train_y)
fit_gpytorch_model(mll)


/Users/macponcho/anaconda3/lib/python3.11/site-packages/botorch/models/utils/assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([1.3547e-09], dtype=torch.float64), std = tensor([1.0235], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


In [170]:
next_query = next_query_via_TurBO(train_x=train_x, train_y=train_y, turbo_state=state)


In [171]:
# Next query with TS
formatted_row_ts = '-'.join(format(x.item(), ".6f") for x in next_query.view(-1))
print("TS:", f"[{formatted_row_ts}]")

# Get the next query point using EI
next_query_ei = generate_batch(state, model=model, X=train_x, Y=train_y, acqf="ei")
formatted_row_ei = '-'.join(format(x.item(), ".6f") for x in next_query_ei.view(-1))
print("EI:", f"[{formatted_row_ei}]")

# Get the next query point using UCB
next_query_ucb = generate_batch(state, model=model, X=train_x, Y=train_y, acqf="ucb")
formatted_row_ucb = '-'.join(format(x.item(), ".6f") for x in next_query_ucb.view(-1))
print("UCB:", f"[{formatted_row_ucb}]")

# Get the next query point using UCB-Z
next_query_ucb0 = generate_batch(state, model=model, X=train_x, Y=train_y, acqf="ucb0")
formatted_row_ucb0 = '-'.join(format(x.item(), ".6f") for x in next_query_ucb0.view(-1))
print("UCB-Z:", f"[{formatted_row_ucb0}]")

TS: [0.400150-0.600044]
EI: [0.353403-0.997757]
UCB: [0.394553-0.480802]
UCB-Z: [0.427731-0.489973]


In [172]:
print(state)

TurboState(dim=2, batch_size=1, length=0.3, length_min=0.01, length_max=1.0, failure_counter=0, failure_tolerance=4, success_counter=0, success_tolerance=5, best_value=tensor(3.3213), restart_triggered=False)


### New observation for the previous query

In [101]:
feedback=0.666666

In [102]:
x_next = next_query  # The new query point from the last run
y_next = torch.tensor(scaler_y.transform([[feedback]]).flatten(), dtype=torch.float64)
print(y_next)

tensor([84.6496], dtype=torch.float64)


In [103]:
new_state = update_state(state, y_next)
print(new_state)

TurboState(dim=2, batch_size=1, length=0.2, length_min=0.01, length_max=1.0, failure_counter=0, failure_tolerance=4, success_counter=1, success_tolerance=5, best_value=84.64962489984764, restart_triggered=False)


In [104]:
# Append the new data to the existing training data
train_x = torch.cat((train_x, x_next.view(1, -1)), dim=0)
train_y = torch.cat((train_y, y_next.view(1)), dim=0)

# Execute the next run to get the next query point
next_query = next_query_via_TurBO(train_x=train_x, train_y=train_y, turbo_state=state)

/Users/macponcho/anaconda3/lib/python3.11/site-packages/botorch/models/utils/assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([3.4393], dtype=torch.float64), std = tensor([16.9469], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [100]:
# Nest query with TS
print(f'Next chose query using TS: {next_query}')

# Get the next query point using EI
next_query_ei = generate_batch(state, model=model, X=train_x, Y=train_y, acqf="ei")
print(f'Next chosen query using EI: {next_query_ei}')

# Get the next query point using UCB
next_query_ucb = generate_batch(state, model=model, X=train_x, Y=train_y, acqf="ucb")
print(f'Next chosen query using UCB: {next_query_ucb}')

# Get the next query point using UCB
next_query_ucb = generate_batch(state, model=model, X=train_x, Y=train_y, acqf="ucb0")
print(f'Next chosen query using UCB-Z: {next_query_ucb}')

Next chose query using TS: tensor([0.3146, 0.3899], dtype=torch.float64, grad_fn=<SelectBackward0>)
Next chosen query using EI: tensor([[0.2788, 0.9771]], requires_grad=True)
Next chosen query using UCB: tensor([[0.3946, 0.4808]], requires_grad=True)
Next chosen query using UCB-Z: tensor([[0.4277, 0.4900]], requires_grad=True)


If you feel TuRBO would help you with the high-dimensional problems in the Capstone, give it a go! There a few questions that may lead to better performance:

1. Maybe you can constraint some of the GPs hyper-parameters for better behaviour?
2. How are you planning to initialize the Turbo State for the first time?

So take the code from this notebook and modify it to your liking!